# 로컬 개발 코드
- 로컬에서 주피터 노트북(Jupyter Notebook), 주피터 랩(JupyterLab) 또는 파이썬(Python)을 이용한다. 
- 사이킷 런(scikit-learn), 텐서플로우(tensorflow), 파이토치(pytorch)를 사용하여 딥러닝 프로그램을 개발한다.
- 파일명: 0_local_Perspecive_NewsRec.ipynb

### 로컬 개발 워크플로우(workflow)  
- 로컬 개발 워크플로우를 다음의 4단계로 분리한다.

1. 데이터 세트 준비(Data Setup)
- 로컬 저장소에서 전처리 및 학습에 필요한 학습 데이터 세트를 준비한다.

2. 데이터 전처리(Data Preprocessing)
- 데이터 세트의 분석 및 정규화(Normalization)등의 전처리를 수행한다.
- 데이터를 모델 학습에 사용할 수 있도록 가공한다.
- 추론과정에서 필요한 경우, 데이터 전처리에 사용된 객체를 meta_data 폴더 아래에 저장한다.

3. 학습 모델 훈련(Train Model)
- 데이터를 훈련에 사용할 수 있도록 가공한 뒤에 학습 모델을 구성한다. 
- 학습 모델을 준비된 데이터 세트로 훈련시킨다.
- 정확도(Accuracy)나 손실(Loss)등 학습 모델의 성능을 검증한다.
- 학습 모델의 성능 검증 후, 학습 모델을 배포한다.
- 배포할 학습 모델을 meta_data 폴더 아래에 저장한다.

4. 추론(Inference)
- 저장된 전처리 객체나 학습 모델 객체를 준비한다.
- 추론에 필요한 테스트 데이터 세트를 준비한다.
- 배포된 학습 모델을 통해 테스트 데이터에 대한 추론을 진행한다. 

In [1]:
pip list

Package                   Version
------------------------- --------------
absl-py                   2.1.0
aiohttp                   3.9.5
aiosignal                 1.3.1
anyio                     4.4.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
async-timeout             4.0.3
attrs                     23.2.0
Babel                     2.15.0
beautifulsoup4            4.12.3
bertopic                  0.16.2
bidict                    0.23.1
bleach                    6.1.0
Bottleneck                1.3.7
Brotli                    1.0.9
bs4                       0.0.2
certifi                   2024.6.2
cffi                      1.16.0
chardet                   4.0.0
charset-normalizer        2.0.4
click                     8.1.7
cmudict                   1.0.24
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.2.1
cycler           

## 뉴스에 반하다
- 이슈성 정보를 다루는 뉴스에서 기존 관점 다른 관점을 제공해 주어 확증 편향을 완화
- 비판적인 사고력과 자기 객관화 능력 및 정보 분별력 등의 능력 신장에 도움을 주는 프로그램 구축을 목표로 함.

### 사용할 데이터
- (기존)매일 22시마다 기사 자동 수집 -> (변경: 고정 DB 사용) 민희진, 밀양사적제재, 북한 오물풍선, 의사 파업, 25만원 총 5개의 키워드로 각각 크롤링한 뉴스 (총 6437개)
- json 파일(기사 데이터), npy 파일 (임베딩 데이터)
- 문서 요약 데이터 (AI-HUB 제공)

### 사용할 요소
- 뉴스 제목, 링크, 기사본문
- 요약모델 훈련을 위해 문서 요약 데이터 사용
- 수집한 기사 본문 바탕으로 요약문장 뽑아내기.

In [1]:
# Imports
import zipfile
import numpy as np
import json
 
import pandas as pd
from sklearn.model_selection import train_test_split


import math
import pandas as pd
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim

from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

from torch.nn.init import xavier_uniform_

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import kss


from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
import faiss

## **1. 데이터셋 준비(Data Setup)**

In [2]:
# dataset.zip 파일 압축 풀기
zip_source_path = './dataset.zip'
zip_target_path = './meta_data'

extract_zip_file = zipfile.ZipFile(zip_source_path)
extract_zip_file.extractall(zip_target_path)
 
extract_zip_file.close()

### 뉴스 데이터셋

In [3]:
my_path = './meta_data/dataset/'

# 뉴스 데이터셋
dataset='news.json'
news_dataset=pd.read_json(my_path+dataset)

In [4]:
# 뉴스 기사 예시 5개
news_dataset.sample(5)

,index,title,link,article
2526,2526,"北이 날린 '오물 풍선', 서울·경기서 90여개 발견",https://n.news.naver.com/mnews/article/366/000...,북한이 지난달 28일 날려 보냈던 오물 풍선을 나흘 만에 다시 띄웠다고 합동참모본부...
1650,1650,북한이 날린 ‘오물 풍선’ 경기·인천 곳곳서 발견…“피해 없어”,https://n.news.naver.com/mnews/article/032/000...,북한이 날린 오물 풍선이 경기도와 인천에서 잇따라 발견돼 경찰과 군 당국이 수거해 ...
3626,3626,"군 ""내일 또 北 '오물 풍선' 살포 가능성… 만지지 말고 신고""",https://n.news.naver.com/mnews/article/656/000...,북한이 최근 도발을 이어가는 가운데 우리 군은 토요일인 1일 북풍이 불 것으로 예고...
1865,1865,"軍 ""北 4차 오물 풍선 310여개""…향후 남북 시나리오는?",https://n.news.naver.com/mnews/article/015/000...,북한이 지난 9일 밤부터 300개가 넘는 오물 풍선을 남쪽에 띄워 보낸 것으로 확인...
6219,6219,與이어 민주당 서울 당선인들 만나는 오세훈,https://n.news.naver.com/mnews/article/020/000...,"30일 시장공관서 회동, 보폭 넓혀여권내 “차기 대권 염두 둔 행보”여권 차기 대선..."


In [5]:
print("( 뉴스기사 수 , 컬럼 수)")
print(news_dataset.shape)

( 뉴스기사 수 , 컬럼 수)
(6437, 4)


### 저장된 뉴스들의 요약 임베딩 데이터

In [6]:
# 뉴스 요약 임베딩 데이터
dataset='summary_embedding.npy'
summary_embedding_dataset=np.load(my_path+dataset)

In [7]:
print("( 뉴스 개수 ,임베딩 차원 )")
summary_embedding_dataset.shape

( 뉴스 개수 ,임베딩 차원 )


(6437, 128)

In [8]:
summary_embedding_dataset[0] # 첫 번째 기사의 요약 임베딩 데이터

array([-2.80979604e-01,  6.20539337e-02, -6.83445334e-02, -1.19693622e-01,
        2.08823726e-01, -1.24537600e-02,  1.18500881e-01,  2.92633682e-01,
        1.27883732e-01, -1.22530796e-01,  1.88773751e-01,  1.58757374e-01,
        7.53097832e-02,  9.50985700e-02,  1.99547485e-02,  1.87469363e-01,
        2.95771599e-01,  9.03890282e-02,  1.81211784e-01,  5.51250339e-01,
        4.68128473e-02, -1.96054339e-01, -2.05385476e-01,  2.59385973e-01,
       -2.40323737e-01, -1.36626616e-01,  4.44304973e-01, -2.42111281e-01,
       -4.77627926e-02,  2.79585384e-02, -3.09917122e-01,  1.79996401e-01,
        1.54761756e-02, -1.70492396e-01, -3.24941099e-01, -8.85238275e-02,
        1.71700209e-01, -1.93334967e-02,  1.79288030e-01, -4.81512427e-01,
        1.27589509e-01,  7.58003443e-05, -2.44617552e-01, -4.17773165e-02,
       -1.59207672e-01,  2.16412395e-02,  2.14116201e-01,  9.37071070e-02,
       -2.15679660e-01, -1.72768682e-01,  2.53318340e-01, -1.23410642e-01,
       -2.44821072e-01, -

### 저장된 뉴스들의 단락 데이터
- index: 기사 번호
- paragraph: 단락

In [9]:
# 저장된 기사들의 단락 데이터
dataset='paragraph_data.json'
paragraph_dataset=pd.read_json(my_path+dataset)

In [10]:
paragraph_dataset

,index,paragraph
0,0,과거에 비해 다소 살이 빠진 듯한 방시혁 하이브 의장의 모습이 소셜미디어(sns) ...
1,0,방 의장은 전날 자신의 인스타그램 계정에 진과 함께 찍은 사진을 공개하며 “성공적인...
2,0,"이날 개최한 팬 이벤트는 진의 전역 후 첫 행사이자, bts의 데뷔 11주년 행사였..."
3,0,방시혁 하이브 의장이 지난달 28일 오후 무함마드 빈 자예드 알 나흐얀 아랍에미리트...
4,0,줄곧 침묵을 유지하던 방 의장은 지난달 17일 법원에 제출한 탄원서를 통해 “한 사...
...,...,...
33648,6435,이 대표는 경남 창원 민주당 경남도당에서 열린 현장 선거대책위원회에서 같은 당 김경...
33649,6435,이 대표는 전날 자신이 내놓은 ‘국민 1인당 25만원씩(총 13조원 추산) 민생회복...
33650,6435,"국민의힘은 세 자녀 등록금 면제 대상은 34만명이고, 들어갈 예산은 1조4500억원..."
33651,6435,"세 자녀 가구에 지원되는 전기요금, 도시가스, 지역난방비 감면을 두 자녀 가구로 확..."


In [11]:
print("( 단락 총 개수 , 컬럼 수)")
paragraph_dataset.shape

( 단락 총 개수 , 컬럼 수)


(33653, 2)

### 저장된 뉴스들의 단락 임베딩 데이터

In [12]:
# 저장된 기사들의 단락 임베딩 데이터
dataset='paragraph_embedding.npy'
paragraph_embedding_dataset=np.load(my_path+dataset)

In [13]:
print("( 단락 총 개수 , 임베딩 차원 )")
paragraph_embedding_dataset.shape

( 단락 총 개수 , 임베딩 차원 )


(33653, 128)

In [14]:
print(paragraph_embedding_dataset[0])# 첫번째 기사의 1번 단락 임베딩 데이터

[-3.27881664e-01  1.14686206e-01 -3.37897390e-02 -1.89114317e-01
  2.53429800e-01  5.00494204e-02  1.96555123e-01  2.81413555e-01
  1.71784639e-01 -1.66616380e-01  2.19282180e-01  1.76256597e-01
 -7.33617367e-03  2.34767750e-01  5.44347195e-03  4.07325588e-02
  1.38898820e-01  1.18467353e-01  3.30788307e-02  5.21330237e-01
  1.35957694e-03 -1.89961404e-01 -1.63521573e-01  2.78955787e-01
 -2.86507934e-01 -2.54717767e-01  4.47581232e-01 -1.88536391e-01
 -8.64003524e-02  1.46690339e-01 -2.52024174e-01  2.27498025e-01
  9.02265385e-02 -9.52593833e-02 -3.43573064e-01 -6.72127530e-02
  1.05423577e-01  1.75513811e-02  1.22265011e-01 -4.44901586e-01
  1.70518607e-01 -1.24860844e-02 -2.04270840e-01 -1.10439554e-01
 -1.18234046e-02  7.24388808e-02  2.57121265e-01  8.17412436e-02
 -1.05895422e-01 -1.48167774e-01  1.48221895e-01  4.64897156e-02
 -5.93772121e-02 -3.49758297e-01 -2.06938535e-01  1.65401340e-01
 -7.27411285e-02 -3.06901336e-02 -1.07786343e-01 -9.71050113e-02
  1.97121829e-01  3.32890

### 요약 모델 훈련을 위한 데이터셋
- AI Hub 문서요약텍스트 데이터셋 (신문 기사 데이터셋)
- https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=97

In [15]:
# train / valid dataset
train_dataset=pd.read_json(my_path+'train_original.json')
valid_dataset=pd.read_json(my_path+'valid_original.json')

In [16]:
valid_dataset

,name,delivery_date,documents
0,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626877', 'category': '정치', 'media_t..."
1,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626896', 'category': '종합', 'media_t..."
2,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340626904', 'category': 'IT,과학', 'medi..."
3,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340627450', 'category': '사회', 'media_t..."
4,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '340627465', 'category': '경제', 'media_t..."
...,...,...,...
30117,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '350851474', 'category': '종합', 'media_t..."
30118,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '350851925', 'category': '경제', 'media_t..."
30119,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '350854748', 'category': '종합', 'media_t..."
30120,문서요약 프로젝트,2020-12-23 12:01:15,"{'id': '350857648', 'category': '종합', 'media_t..."


## **2. 데이터 전처리 (Data Preprocessing)**

### 데이터 준비 (Preparing Data)
요약모델을 위한 데이터셋을 훈련에 사용할 수 있는 형태로 변형

- 결측 데이터 삭제
- 훈련/테스트 데이터셋을 9.5:0.5 비율로 나눔
- 기존 데이터셋이었던 Bflysoft에서 제공한 뉴스 데이터가 비공개처리 되어 대체 데이터셋인 AI-HUB 문서 요약 데이터셋을 기존 데이터셋처럼 변형

#### 결측 데이터 삭제

In [17]:
train_dataset=train_dataset.dropna()
print("train_dataset 수:"+str(len(train_dataset)))

valid_dataset=valid_dataset.dropna()
print("valid_dataset 수:"+str(len(valid_dataset)))

train_dataset 수:243983
valid_dataset 수:30122


# 훈련 & 평가 데이터셋 생성

훈련 데이터셋에서 0.05를 valid 데이터셋으로 사용 (https://github.com/KPFBERT/kpfbertsum/blob/main/kpfbert_summary.ipynb)

In [18]:
train_df, val_df=train_test_split(train_dataset, test_size=0.05)
test_df=valid_dataset
train_df=train_df.reset_index(drop=True)
val_df=val_df.reset_index(drop=True)

In [19]:
print("( 훈련 데이터 개수 , 검증 데이터 수, 테스트 데이터 수)")
print(train_df.shape, val_df.shape, test_df.shape)

( 훈련 데이터 개수 , 검증 데이터 수, 테스트 데이터 수)
(231783, 3) (12200, 3) (30122, 3)


#### 기존 Bflysoft-뉴스기사 데이터셋에 맞춰 변환
- 기존에 Bflysoft-뉴스기사 데이터셋에 맞춰 작성된 코드이나 해당 데이터셋이 비공개되어 AI-HUB 데이터셋을 변형시켜 사용

In [15]:
def preprocess_data(data):
    outs = []
    for doc in data['documents']:
        line = []
        line.append(doc['media_name'])
        line.append(doc['id'])
        para = []
        for sent in doc['text']:
            for s in sent:
                para.append(s['sentence'])
        line.append(para)
        line.append(doc['abstractive'][0])
        line.append(doc['extractive'])
        a = doc['extractive']
        if a[0] == None or a[1] == None or a[2] == None:
            continue
        outs.append(line)

    outs_df = pd.DataFrame(outs)
    outs_df.columns = ['media', 'id', 'article_original', 'abstractive', 'extractive']
    return outs_df

In [21]:
train_df = preprocess_data(train_df)
train_df.head(1)

,media,id,article_original,abstractive,extractive
0,전라일보,338837772,"[최홍은기자l hiimnews@, ▲ 문재인 대통령이 25일 오후 청와대에서 열린 ...","문재인 대통령은 25일 “특권층의 불법적 행위와 외압에 의한 부실수사, 권력의 비호...","[2, 4, 6]"


In [22]:
test_df = preprocess_data(test_df)
test_df.head(1)

,media,id,article_original,abstractive,extractive
0,한국경제,340626877,"[[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, ...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...,"[0, 1, 3]"


In [23]:
val_df = preprocess_data(val_df)
val_df.head(1)

,media,id,article_original,abstractive,extractive
0,아시아경제,366155197,"[김재원 국회 예결위원장과 맥주를 마시고 있다., 강기정 청와대 정무수석이 자유한국...",강기정 수석은 예결위 전체회의 참석을 위해 국회를 찾았지만 한국당 의원들은 강 수석...,"[3, 6, 7]"


## **3. 학습 모델 훈련 (Train Model)**
요약 모델 훈련

In [24]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

Global seed set to 42


42

In [16]:
BERT_MODEL_NAME = 'jinmang2/kpfbert'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [17]:
MAX_TOKEN_COUNT = 512
N_EPOCHS = 10
BATCH_SIZE = 4

In [18]:
class SummDataset(Dataset):

    def __init__(
        self, 
        data: pd.DataFrame, 
        tokenizer: BertTokenizer, 
        max_token_len: int = 512
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        tokenlist = []
        for sent in data_row.article_original:
            tokenlist.append(tokenizer(
                text = sent,
                add_special_tokens = True)) #, # Add '[CLS]' and '[SEP]'
    
        src = [] # 토크나이징 된 전체 문단
        labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
        segs = []  #각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
        clss = []  #[CLS]토큰의 포지션값을 지정

        odd = 0
        for tkns in tokenlist:
            if odd > 1 : odd = 0
            clss = clss + [len(src)]
            src = src + tkns['input_ids']
            segs = segs + [odd] * len(tkns['input_ids'])
            if tokenlist.index(tkns) in data_row.extractive :
                labels = labels + [1]
            else:
                labels = labels + [0]
            odd += 1
        
            #truncation
            if len(src) == MAX_TOKEN_COUNT:
                break
            elif len(src) > MAX_TOKEN_COUNT:
                src = src[:self.max_token_len - 1] + [src[-1]]
                segs = segs[:self.max_token_len]
                break
    
        #padding
        if len(src) < MAX_TOKEN_COUNT:
            src = src + [0]*(self.max_token_len - len(src))
            segs = segs + [0]*(self.max_token_len - len(segs))
            
        if len(clss) < MAX_TOKEN_COUNT:
            clss = clss + [-1]*(self.max_token_len - len(clss))
        if len(labels) < MAX_TOKEN_COUNT:
            labels = labels + [0]*(self.max_token_len - len(labels))

        return dict(
            src = torch.tensor(src),
            segs = torch.tensor(segs),
            clss = torch.tensor(clss),
            labels= torch.FloatTensor(labels)
        )
class SummDataModule(pl.LightningDataModule):

    def __init__(self, train_df, test_df, val_df, tokenizer, batch_size=1, max_token_len=512):
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.val_df = val_df
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = SummDataset(
            self.train_df,
            self.tokenizer,
            self.max_token_len
        )

        self.test_dataset = SummDataset(
            self.test_df,
            self.tokenizer,
            self.max_token_len
        )
    
        self.val_dataset = SummDataset(
            self.val_df,
            self.tokenizer,
            self.max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )

    def test_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=0 # windows는 0으로 고정해야 에러 안난다. num_workers=2
        )
data_module = SummDataModule(
  train_df,
  test_df,  
  val_df,
  tokenizer,
  batch_size=BATCH_SIZE,
  max_token_len=MAX_TOKEN_COUNT
)

NameError: name 'train_df' is not defined

#### MODEL
kpfBERT를 pretrained_bert로 불러와서 후처리 레이어를 추가하여 문장추출 모델을 만든다.

In [20]:
class PositionalEncoding(nn.Module):

    def __init__(self, dropout, dim, max_len=5000):
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp((torch.arange(0, dim, 2, dtype=torch.float) *
                              -(math.log(10000.0) / dim)))
        pe[:, 0::2] = torch.sin(position.float() * div_term)
        pe[:, 1::2] = torch.cos(position.float() * div_term)
        pe = pe.unsqueeze(0)
        super(PositionalEncoding, self).__init__()
        self.register_buffer('pe', pe)
        self.dropout = nn.Dropout(p=dropout)
        self.dim = dim

    def forward(self, emb, step=None):
        emb = emb * math.sqrt(self.dim)
        if (step):
            emb = emb + self.pe[:, step][:, None, :]

        else:
            emb = emb + self.pe[:, :emb.size(1)]
        emb = self.dropout(emb)
        return emb

    def get_emb(self, emb):
        return self.pe[:, :emb.size(1)]
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, heads, d_ff, dropout):
        super(TransformerEncoderLayer, self).__init__()

        self.self_attn = MultiHeadedAttention(
            heads, d_model, dropout=dropout)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, iter, query, inputs, mask):
        if (iter != 0):
            input_norm = self.layer_norm(inputs)
        else:
            input_norm = inputs

        mask = mask.unsqueeze(1)
        context = self.self_attn(input_norm, input_norm, input_norm,
                                 mask=mask)
        out = self.dropout(context) + inputs
        return self.feed_forward(out)
class ExtTransformerEncoder(nn.Module):
    def __init__(self, hidden_size=768, d_ff=2048, heads=8, dropout=0.2, num_inter_layers=2):
        super(ExtTransformerEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_inter_layers = num_inter_layers
        self.pos_emb = PositionalEncoding(dropout, hidden_size)
        self.transformer_inter = nn.ModuleList(
            [TransformerEncoderLayer(hidden_size, heads, d_ff, dropout)
            for _ in range(num_inter_layers)])
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(hidden_size, eps=1e-6)
        self.wo = nn.Linear(hidden_size, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, top_vecs, mask):
        """ See :obj:`EncoderBase.forward()`"""

        batch_size, n_sents = top_vecs.size(0), top_vecs.size(1)
        pos_emb = self.pos_emb.pe[:, :n_sents]
        x = top_vecs * mask[:, :, None].float()
        x = x + pos_emb

        for i in range(self.num_inter_layers):
            x = self.transformer_inter[i](i, x, x, ~mask) 

        x = self.layer_norm(x)
        sent_scores = self.sigmoid(self.wo(x))
        sent_scores = sent_scores.squeeze(-1) * mask.float()

        return sent_scores
class PositionwiseFeedForward(nn.Module):
    """ A two-layer Feed-Forward-Network with residual layer norm.

    Args:
        d_model (int): the size of input for the first-layer of the FFN.
        d_ff (int): the hidden layer size of the second-layer
            of the FNN.
        dropout (float): dropout probability in :math:`[0, 1)`.
    """

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def gelu(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


    def forward(self, x):
        inter = self.dropout_1(self.gelu(self.w_1(self.layer_norm(x))))
        output = self.dropout_2(self.w_2(inter))
        return output + x
class MultiHeadedAttention(nn.Module):
    """
    Multi-Head Attention module from
    "Attention is All You Need"
    :cite:`DBLP:journals/corr/VaswaniSPUJGKP17`.

    Similar to standard `dot` attention but uses
    multiple attention distributions simulataneously
    to select relevant items.

    .. mermaid::

       graph BT
          A[key]
          B[value]
          C[query]
          O[output]
          subgraph Attn
            D[Attn 1]
            E[Attn 2]
            F[Attn N]
          end
          A --> D
          C --> D
          A --> E
          C --> E
          A --> F
          C --> F
          D --> O
          E --> O
          F --> O
          B --> O

    Also includes several additional tricks.

    Args:
       head_count (int): number of parallel heads
       model_dim (int): the dimension of keys/values/queries,
           must be divisible by head_count
       dropout (float): dropout parameter
    """

    def __init__(self, head_count, model_dim, dropout=0.1, use_final_linear=True):
        assert model_dim % head_count == 0
        self.dim_per_head = model_dim // head_count
        self.model_dim = model_dim

        super(MultiHeadedAttention, self).__init__()
        self.head_count = head_count

        self.linear_keys = nn.Linear(model_dim,
                                     head_count * self.dim_per_head)
        self.linear_values = nn.Linear(model_dim,
                                       head_count * self.dim_per_head)
        self.linear_query = nn.Linear(model_dim,
                                      head_count * self.dim_per_head)
        self.softmax = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)
        self.use_final_linear = use_final_linear
        if (self.use_final_linear):
            self.final_linear = nn.Linear(model_dim, model_dim)

    def forward(self, key, value, query, mask=None,
                layer_cache=None, type=None, predefined_graph_1=None):
        """
        Compute the context vector and the attention vectors.

        Args:
           key (`FloatTensor`): set of `key_len`
                key vectors `[batch, key_len, dim]`
           value (`FloatTensor`): set of `key_len`
                value vectors `[batch, key_len, dim]`
           query (`FloatTensor`): set of `query_len`
                 query vectors  `[batch, query_len, dim]`
           mask: binary mask indicating which keys have
                 non-zero attention `[batch, query_len, key_len]`
        Returns:
           (`FloatTensor`, `FloatTensor`) :

           * output context vectors `[batch, query_len, dim]`
           * one of the attention vectors `[batch, query_len, key_len]`
        """

        batch_size = key.size(0)
        dim_per_head = self.dim_per_head
        head_count = self.head_count
        key_len = key.size(1)
        query_len = query.size(1)

        def shape(x):
            """  projection """
            return x.view(batch_size, -1, head_count, dim_per_head) \
                .transpose(1, 2)

        def unshape(x):
            """  compute context """
            return x.transpose(1, 2).contiguous() \
                .view(batch_size, -1, head_count * dim_per_head)

        # 1) Project key, value, and query.
        if layer_cache is not None:
            if type == "self":
                query, key, value = self.linear_query(query), \
                                    self.linear_keys(query), \
                                    self.linear_values(query)

                key = shape(key)
                value = shape(value)

                if layer_cache is not None:
                    device = key.device
                    if layer_cache["self_keys"] is not None:
                        key = torch.cat(
                            (layer_cache["self_keys"].to(device), key),
                            dim=2)
                    if layer_cache["self_values"] is not None:
                        value = torch.cat(
                            (layer_cache["self_values"].to(device), value),
                            dim=2)
                    layer_cache["self_keys"] = key
                    layer_cache["self_values"] = value
            elif type == "context":
                query = self.linear_query(query)
                if layer_cache is not None:
                    if layer_cache["memory_keys"] is None:
                        key, value = self.linear_keys(key), \
                                     self.linear_values(value)
                        key = shape(key)
                        value = shape(value)
                    else:
                        key, value = layer_cache["memory_keys"], \
                                     layer_cache["memory_values"]
                    layer_cache["memory_keys"] = key
                    layer_cache["memory_values"] = value
                else:
                    key, value = self.linear_keys(key), \
                                 self.linear_values(value)
                    key = shape(key)
                    value = shape(value)
        else:
            key = self.linear_keys(key)
            value = self.linear_values(value)
            query = self.linear_query(query)
            key = shape(key)
            value = shape(value)

        query = shape(query)

        key_len = key.size(2)
        query_len = query.size(2)

        # 2) Calculate and scale scores.
        query = query / math.sqrt(dim_per_head)
        scores = torch.matmul(query, key.transpose(2, 3))

        if mask is not None:
            mask = mask.unsqueeze(1).expand_as(scores)
            scores = scores.masked_fill(mask, -1e18) # how can i fix it to use fp16...

        # 3) Apply attention dropout and compute context vectors.

        attn = self.softmax(scores)

        if (not predefined_graph_1 is None):
            attn_masked = attn[:, -1] * predefined_graph_1
            attn_masked = attn_masked / (torch.sum(attn_masked, 2).unsqueeze(2) + 1e-9)

            attn = torch.cat([attn[:, :-1], attn_masked.unsqueeze(1)], 1)

        drop_attn = self.dropout(attn)
        if (self.use_final_linear):
            context = unshape(torch.matmul(drop_attn, value))
            output = self.final_linear(context)
            return output
        else:
            context = torch.matmul(drop_attn, value)
            return context

class Summarizer(pl.LightningModule):

    def __init__(self, n_training_steps=None, n_warmup_steps=None):
        super().__init__()
        self.max_pos = 512
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME) #, return_dict=True)
        self.ext_layer = ExtTransformerEncoder()
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.loss = nn.BCELoss(reduction='none')
    
        for p in self.ext_layer.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

    def forward(self, src, segs, clss, labels=None): #, input_ids, attention_mask, labels=None):
        
        mask_src = ~(src == 0) #1 - (src == 0)
        mask_cls = ~(clss == -1) #1 - (clss == -1)

        top_vec = self.bert(src, token_type_ids=segs, attention_mask=mask_src)
        top_vec = top_vec.last_hidden_state
        
        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
        sents_vec = sents_vec * mask_cls[:, :, None].float()

        sent_scores = self.ext_layer(sents_vec, mask_cls).squeeze(-1)
        
        loss = 0
        if labels is not None:
            loss = self.loss(sent_scores, labels)
            
            loss = (loss * mask_cls.float()).sum() / len(labels)
        
        return loss, sent_scores
    
    def step(self, batch):

        src = batch['src']
        if len(batch['labels']) > 0 :
            labels = batch['labels']
        else:
            labels = None
        segs = batch['segs']
        clss = batch['clss']
        
        loss, sent_scores = self(src, segs, clss, labels)    
        
        return loss, sent_scores, labels

    def training_step(self, batch, batch_idx):

        loss, sent_scores, labels = self.step(batch)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def validation_step(self, batch, batch_idx):
        
        loss, sent_scores, labels = self.step(batch)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def test_step(self, batch, batch_idx):
        
        loss, sent_scores, labels = self.step(batch)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        
        return {"loss": loss, "predictions": sent_scores, "labels": labels}

    def acc_loss(self, outputs):
        total_loss = 0
        hit_cnt = 0
        for outp in outputs:
            labels = outp['labels'].cpu()
            predictions, idxs = outp['predictions'].cpu().sort()
            loss = outp['loss'].cpu()
            for label, idx in zip(labels, idxs):
                for i in range(1,3):
                    if label[idx[-i-1]] == 1 : 
                        hit_cnt += 1

            total_loss += loss
            
        avg_loss = total_loss / len(outputs)
        acc = hit_cnt / (3*len(outputs)*len(labels))
        
        return acc, avg_loss
        
    def training_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('acc:', acc, 'avg_loss:', avg_loss)
        
        self.log('avg_train_loss', avg_loss, prog_bar=True, logger=True)

    def validation_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('val_acc:', acc, 'avg_val_loss:', avg_loss)
        
        self.log('avg_val_loss', avg_loss, prog_bar=True, logger=True)

    def test_epoch_end(self, outputs):
        
        acc, avg_loss = self.acc_loss(outputs)
        
        print('test_acc:', acc, 'avg_test_loss:', avg_loss)
        
        self.log('avg_test_loss', avg_loss, prog_bar=True, logger=True)

        return
        
    def configure_optimizers(self):
        
        optimizer = AdamW(self.parameters(), lr=2e-5)

        steps_per_epoch=len(train_df) // BATCH_SIZE
        total_training_steps = steps_per_epoch * N_EPOCHS
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=steps_per_epoch,
            num_training_steps=total_training_steps
        )

        return dict(
            optimizer=optimizer,
            lr_scheduler=dict(
                scheduler=scheduler,
                interval='step'
            )
        )

### 모델 컴파일 및 학습 (Compile and Train Model)

- Fine Tuning
- 가장 최적의 결과를 낸 모델을 checkpoints/best-checkpoint.ckpt 로 저장
- lightning_logs/kpfBERT_Summary/version_x/ 하위에 fine-tuning 로그가 저장됨, x는 실행순서 번호

In [21]:
model = Summarizer()

Some weights of BertModel were not initialized from the model checkpoint at jinmang2/kpfbert and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
#windows
!rmdir /s /q lightning_logs
!rmdir /s /q  checkpoints

지정된 파일을 찾을 수 없습니다.


In [31]:

%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

Reusing TensorBoard on port 6006 (pid 16100), started 2 days, 22:56:35 ago. (Use '!kill 16100' to kill it.)

In [22]:

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="avg_val_loss",
    mode="min"
)

In [23]:
logger = TensorBoardLogger("lightning_logs", name="kpfBERT_Summary")


In [24]:
early_stopping_callback = EarlyStopping(monitor='avg_val_loss', patience=3)


In [25]:
trainer = pl.Trainer(
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    callbacks=[early_stopping_callback],
    max_epochs=N_EPOCHS,
    gpus=0,
    #progress_bar_refresh_rate=30
#     precision=16, #소스 수정 또는 패키지 재설치 필요... 런타임 에러.
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [37]:
trainer.fit(model, data_module)

C:\Users\hyj_0\anaconda3\envs\env\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name      | Type                  | Params
----------------------------------------------------
0 | bert      | BertModel             | 114 M 
1 | ext_layer | ExtTransformerEncoder | 11.0 M
2 | loss      | BCELoss               | 0     
----------------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.230   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

val_acc: 0.16666666666666666 avg_val_loss: tensor(24.4402)


Training: 0it [00:00, ?it/s]

C:\Users\hyj_0\AppData\Roaming\Python\Python39\site-packages\pytorch_lightning\utilities\distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Epoch 0, global step 1133: avg_val_loss reached 24.44023 (best 24.44023), saving model to "checkpoints\best-checkpoint.ckpt" as top 1


1

### 모델 평가 (Evaluate Model)

In [38]:
trainer.test()

Testing: 0it [00:00, ?it/s]

KeyboardInterrupt: 

## **4. 추론 (Inference)**

1. 훈련한 요약 모델을 바탕으로 테스트 뉴스 데이터의 요약본 추출 및 임베딩
3. 피어슨 상관관계를 이용하여 기존 뉴스 요약 임베딩 데이터와 비교 (threshold=0.55)
4. 높은 유사도를 가진 뉴스 데이터(최대 100개)를 추출
5. 테스트 뉴스의 단락 데이터 생성 및 임베딩
6. 저장된 단락 임베딩 데이터와 테스트 뉴스의 단락 데이터를 이용하여 BERTopic으로 클러스터링 진행
7. 클러스터링 결과 바탕으로 다른 뉴스 3개 출력

##### 테스트 데이터셋 불러오기

In [26]:
# dataset.zip 파일을 dataset 폴더에 압축을 풀어준다.
zip_source_path = './test_dataset.zip'
zip_target_path = './meta_data'

extract_zip_file = zipfile.ZipFile(zip_source_path)
extract_zip_file.extractall(zip_target_path)
 
extract_zip_file.close()

In [27]:
my_path='./meta_data/test_dataset/'

test_name='test.json'
with open(my_path+test_name, encoding='utf8') as f:
    test_dataset = json.load(f)

In [28]:
target_article=test_dataset['article']
target_link=test_dataset['link']

#### 테스트 뉴스의 본문 

In [29]:
target_article

'당정에 “구체안 신속 논의” 제안연금개혁안 이은 ‘실용 정치’ 포석野, 30일 관련법안 당론 채택 예고더불어민주당 이재명 대표가 민생회복지원금과 관련해 고수해오던 ‘보편 지원’ 주장을 내려놨다. 정부여당을 향해 “차등 지원도 수용하겠다”며 이른 시일 내 협의하자고 제안한 것이다. 하지만 여당은 ‘차등 지원’을 전제로 하더라도 여전히 민생회복지원금 자체에 부정적인 모습이다. 민주당은 4·10 총선 때부터 ‘전 국민 25만원 지급’을 골자로 한 민생회복지원금 제도 시행을 촉구해왔다.이 대표는 29일 당 최고위원회의에서 “민생회복지원금은 소득 지원 효과도 있지만 지역·지방 소비를 늘려서 경제를 활성화하는 경제 정책이다. 반드시 지원해야 한다”며 “골목경제가 살아나면 정부여당 지지율도 올라가고 좋지 않냐”고 말했다.끝까지 대결 정치더불어민주당 이재명 대표(오른쪽)가 29일 서울 여의도 국회에서 열린 최고위원회의에서 윤석열 대통령과 이종섭 전 국방부 장관의 통화 사실을 보도한 자료를 보며 정청래 최고위원과 대화하고 있다. 남제현 선임기자이 대표는 윤석열 대통령과 정부여당을 향해 “우리가 지원금을 반드시 (전 국민에게) 똑같이 지급하라는 주장을 더 이상 하지 않겠다”며 “우리가 지향하는 가치는 보편 지원에 있긴 하지만 굳이 어렵다면 차등 지원도 수용하겠다”고 했다. 그러면서 고소득층 대상 매칭 지원 등 구체적 방안도 제시했다. 이 대표는 그간 고수해온 ‘보편 지원’ 원칙을 철회한 데 대해 “안 하는 것보다 낫기 때문”이라며 “우리가 양보할 테니 경기도 살리고 민생도 살리는 정책을 수용해주시고, 구체적 내용은 신속하게 만나서 협의하면 좋겠다”고 했다.이번 제안은 이 대표의 ‘실용 정치’ 행보의 일환으로 해석된다. 연금개혁안에 이어 민생 관련 사안에 대한 책임정당 모습을 선점하기 위한 포석이기도 하다. 민주당 한민수 대변인은 이날 이 대표 제안에 대해 “이 대표가 연금개혁에 이어 다시 한 번 양보한 안을 냈다”며 “민생과 경제를 책임지는 대통령과 정부여당의 답이 있을 것”

### 요약모델 이용해서 테스트 뉴스의 요약본 추출

In [30]:
# 훈련된 요약 모델 로드
trained_model = Summarizer.load_from_checkpoint(
    'checkpoints/best-checkpoint.ckpt',
    strict=False
)
trained_model.eval()
trained_model.freeze()

# 문장 분리 함수
def data_process(text):
    # 문장 분리 하고,
    sents = kss.split_sentences(text)

    # 데이터 가공하고,
    tokenlist = []
    for sent in sents:
        tokenlist.append(tokenizer(
            text=sent,
            add_special_tokens=True))  # , # Add '[CLS]' and '[SEP]'

    src = []  # 토크나이징 된 전체 문단
    labels = []  # 요약문에 해당하면 1, 아니면 0으로 문장수 만큼 생성
    segs = []  # 각 토큰에 대해 홀수번째 문장이면 0, 짝수번째 문장이면 1을 매핑
    clss = []  # [CLS]토큰의 포지션값을 지정

    odd = 0

    for tkns in tokenlist:

        if odd > 1:
            odd = 0
        clss = clss + [len(src)]
        src = src + tkns['input_ids']
        segs = segs + [odd] * len(tkns['input_ids'])
        odd += 1

        # truncation
        if len(src) == MAX_TOKEN_COUNT:
            break
        elif len(src) > MAX_TOKEN_COUNT:
            src = src[:MAX_TOKEN_COUNT - 1] + [src[-1]]
            segs = segs[:MAX_TOKEN_COUNT]
            break

    # padding
    if len(src) < MAX_TOKEN_COUNT:
        src = src + [0]*(MAX_TOKEN_COUNT - len(src))
        segs = segs + [0]*(MAX_TOKEN_COUNT - len(segs))

    if len(clss) < MAX_TOKEN_COUNT:
        clss = clss + [-1]*(MAX_TOKEN_COUNT - len(clss))

    return dict(
        sents=sents,  # 정답 출력을 위해...
        src=torch.tensor(src),
        segs=torch.tensor(segs),
        clss=torch.tensor(clss),
    )

# 요약본 추출 함수
def summarize_test(text):
    data = data_process(text.replace('\n', ''))

    # trained_model에 넣어 결과값 반환
    _, rtn = trained_model(data['src'].unsqueeze(
        0), data['segs'].unsqueeze(0), data['clss'].unsqueeze(0))
    rtn = rtn.squeeze()

    # 예측 결과값을 받기 위한 프로세스
    rtn_sort, idx = rtn.sort(descending=True)

    rtn_sort = rtn_sort.tolist()
    idx = idx.tolist()

    end_idx = rtn_sort.index(0)

    rtn_sort = rtn_sort[:end_idx]
    idx = idx[:end_idx]

    if len(idx) > 3:
        rslt = idx[:3]
    else:
        rslt = idx

    summ = []
    for i, r in enumerate(rslt):
        summ.append(data['sents'][r])

    return summ

# 요약본 결과 반환
def summarize_article(target_article):
    target_summary = summarize_test(target_article)
    return target_summary


Some weights of BertModel were not initialized from the model checkpoint at jinmang2/kpfbert and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
target_summary=summarize_article(target_article)
target_summary=" ".join(target_summary)

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



In [32]:
target_summary

'정부여당을 향해 “차등 지원도 수용하겠다”며 이른 시일 내 협의하자고 제안한 것이다. 당정에 “구체안 신속 논의” 제안연금개혁안 이은 ‘실용 정치’ 포석野, 30일 관련법안 당론 채택 예고더불어민주당 이재명 대표가 민생회복지원금과 관련해 고수해오던 ‘보편 지원’ 주장을 내려놨다. 하지만 여당은 ‘차등 지원’을 전제로 하더라도 여전히 민생회복지원금 자체에 부정적인 모습이다.'

### 추출한 요약모델 임베딩

In [33]:
model = SentenceTransformer('bongsoo/kpf-sbert-128d-v1')  # 임베딩 모델
model.max_seq_length = 512

In [34]:
target_summary_embedding = model.encode(target_summary,normalize_embeddings=True)

In [35]:
target_summary_embedding.shape

(128,)

### 테스트 요약 임베딩과 수집 기사 요약 임베딩 유사도 비교

In [36]:
# 피어슨 상관계수 구하기
def pearson_similarity(a, b):
    return np.dot((a-np.mean(a)), (b-np.mean(b)))/((np.linalg.norm(a-np.mean(a)))*(np.linalg.norm(b-np.mean(b))))

In [37]:
# 피어슨 상관계수 기반으로 계산
threshold = 0.55  # 최소 유사도 threshold
similar_list = []
for i in range(len(summary_embedding_dataset)):
    similarity = pearson_similarity(target_summary_embedding, summary_embedding_dataset[i])

    if similarity > threshold:
        similar_list.append((similarity, i)) # threshold 이상이면 유사한 기사 리스트에 추가

# 유사도 기준 내림차순 정렬
sorted_similar_list = sorted(similar_list, key=lambda x: x[0], reverse=True)

# 100개 이상이면 100개만 추려서 반환
if len(similar_list) > 100:
    similar_index_list=[item[1] for item in sorted_similar_list[:100]]

# 100개 이하면 모두 반환
else:
    similar_index_list=[item[1] for item in sorted_similar_list]

In [58]:
print(similar_index_list) # 유사한 기사들의 index

[5523, 5532, 5525, 5544, 5545, 5530, 5751, 5540, 5542, 5992, 5619, 6280, 5943, 5849, 5527, 6293, 6048, 5807, 5538, 5750, 5990, 6120, 5938, 5721, 5533, 5675, 5594, 6315, 3328, 434, 6099, 6246, 5732, 5976, 5697, 5801, 5664, 6376, 3435, 6124, 5554, 6140]


### 테스트 기사의 단락 생성
- 1 단락 = 3 문장

In [39]:
def split_into_paragraphs(article, sentences_per_paragraph=3):
    sentences = kss.split_sentences(article)
    paragraphs = []
    paragraph = []

    for sentence in sentences:
        if len(sentence) > 20:
            # 보통 한 줄에 20자 정도 넘어가야 유의미한 정보가 포함된 문장임
            paragraph.append(sentence)
        if len(paragraph) == sentences_per_paragraph:  # 3줄 이상이면
            paragraphs.append(" ".join(paragraph))  # 3줄을 하나로 합치기
            paragraph = []

        # 남아있는 문장들 중 20자가 넘어가면 단락으로 추가
    if paragraph and len(paragraph) > 20:
        paragraphs.append(" ".join(paragraph))

    return paragraphs  # 단락 데이터 반환

In [40]:
target_paragraphs=split_into_paragraphs(target_article)

In [41]:
target_paragraphs

['당정에 “구체안 신속 논의” 제안연금개혁안 이은 ‘실용 정치’ 포석野, 30일 관련법안 당론 채택 예고더불어민주당 이재명 대표가 민생회복지원금과 관련해 고수해오던 ‘보편 지원’ 주장을 내려놨다. 정부여당을 향해 “차등 지원도 수용하겠다”며 이른 시일 내 협의하자고 제안한 것이다. 하지만 여당은 ‘차등 지원’을 전제로 하더라도 여전히 민생회복지원금 자체에 부정적인 모습이다.',
 '민주당은 4·10 총선 때부터 ‘전 국민 25만원 지급’을 골자로 한 민생회복지원금 제도 시행을 촉구해왔다. 이 대표는 29일 당 최고위원회의에서 “민생회복지원금은 소득 지원 효과도 있지만 지역·지방 소비를 늘려서 경제를 활성화하는 경제 정책이다. 반드시 지원해야 한다”며 “골목경제가 살아나면 정부여당 지지율도 올라가고 좋지 않냐”고 말했다.',
 '끝까지 대결 정치더불어민주당 이재명 대표(오른쪽)가 29일 서울 여의도 국회에서 열린 최고위원회의에서 윤석열 대통령과 이종섭 전 국방부 장관의 통화 사실을 보도한 자료를 보며 정청래 최고위원과 대화하고 있다. 남제현 선임기자이 대표는 윤석열 대통령과 정부여당을 향해 “우리가 지원금을 반드시 (전 국민에게) 똑같이 지급하라는 주장을 더 이상 하지 않겠다”며 “우리가 지향하는 가치는 보편 지원에 있긴 하지만 굳이 어렵다면 차등 지원도 수용하겠다”고 했다. 그러면서 고소득층 대상 매칭 지원 등 구체적 방안도 제시했다.',
 '이 대표는 그간 고수해온 ‘보편 지원’ 원칙을 철회한 데 대해 “안 하는 것보다 낫기 때문”이라며 “우리가 양보할 테니 경기도 살리고 민생도 살리는 정책을 수용해주시고, 구체적 내용은 신속하게 만나서 협의하면 좋겠다”고 했다. 이번 제안은 이 대표의 ‘실용 정치’ 행보의 일환으로 해석된다. 연금개혁안에 이어 민생 관련 사안에 대한 책임정당 모습을 선점하기 위한 포석이기도 하다.',
 '민주당 한민수 대변인은 이날 이 대표 제안에 대해 “이 대표가 연금개혁에 이어 다시 한 번 양보한 안을 냈다”며 “민생과 경제를 책임지는

#### pandas 형식으로 만들기

In [42]:
target_paragraph_data = []
for data in target_paragraphs:
    target_paragraph_data.append([-1]+[data])

In [43]:
target_paragraph_data

[[-1,
  '당정에 “구체안 신속 논의” 제안연금개혁안 이은 ‘실용 정치’ 포석野, 30일 관련법안 당론 채택 예고더불어민주당 이재명 대표가 민생회복지원금과 관련해 고수해오던 ‘보편 지원’ 주장을 내려놨다. 정부여당을 향해 “차등 지원도 수용하겠다”며 이른 시일 내 협의하자고 제안한 것이다. 하지만 여당은 ‘차등 지원’을 전제로 하더라도 여전히 민생회복지원금 자체에 부정적인 모습이다.'],
 [-1,
  '민주당은 4·10 총선 때부터 ‘전 국민 25만원 지급’을 골자로 한 민생회복지원금 제도 시행을 촉구해왔다. 이 대표는 29일 당 최고위원회의에서 “민생회복지원금은 소득 지원 효과도 있지만 지역·지방 소비를 늘려서 경제를 활성화하는 경제 정책이다. 반드시 지원해야 한다”며 “골목경제가 살아나면 정부여당 지지율도 올라가고 좋지 않냐”고 말했다.'],
 [-1,
  '끝까지 대결 정치더불어민주당 이재명 대표(오른쪽)가 29일 서울 여의도 국회에서 열린 최고위원회의에서 윤석열 대통령과 이종섭 전 국방부 장관의 통화 사실을 보도한 자료를 보며 정청래 최고위원과 대화하고 있다. 남제현 선임기자이 대표는 윤석열 대통령과 정부여당을 향해 “우리가 지원금을 반드시 (전 국민에게) 똑같이 지급하라는 주장을 더 이상 하지 않겠다”며 “우리가 지향하는 가치는 보편 지원에 있긴 하지만 굳이 어렵다면 차등 지원도 수용하겠다”고 했다. 그러면서 고소득층 대상 매칭 지원 등 구체적 방안도 제시했다.'],
 [-1,
  '이 대표는 그간 고수해온 ‘보편 지원’ 원칙을 철회한 데 대해 “안 하는 것보다 낫기 때문”이라며 “우리가 양보할 테니 경기도 살리고 민생도 살리는 정책을 수용해주시고, 구체적 내용은 신속하게 만나서 협의하면 좋겠다”고 했다. 이번 제안은 이 대표의 ‘실용 정치’ 행보의 일환으로 해석된다. 연금개혁안에 이어 민생 관련 사안에 대한 책임정당 모습을 선점하기 위한 포석이기도 하다.'],
 [-1,
  '민주당 한민수 대변인은 이날 이 대표 제안에 대해 “이 대표가 연금

In [44]:
target_paragraph_data = pd.DataFrame(data=target_paragraph_data, columns=['index', 'paragraph'])

In [45]:
target_paragraph_data

,index,paragraph
0,-1,"당정에 “구체안 신속 논의” 제안연금개혁안 이은 ‘실용 정치’ 포석野, 30일 관련..."
1,-1,민주당은 4·10 총선 때부터 ‘전 국민 25만원 지급’을 골자로 한 민생회복지원금...
2,-1,끝까지 대결 정치더불어민주당 이재명 대표(오른쪽)가 29일 서울 여의도 국회에서 열...
3,-1,이 대표는 그간 고수해온 ‘보편 지원’ 원칙을 철회한 데 대해 “안 하는 것보다 낫...
4,-1,민주당 한민수 대변인은 이날 이 대표 제안에 대해 “이 대표가 연금개혁에 이어 다시...
5,-1,끝까지 대결 정치국민의힘 추경호 원내대표(오른쪽)가 29일 서울 여의도 국회에서 열...


#### 위에서 구한 유사한 기사 인덱스를 이용해서 유사한 기사 데이터만 가져오기

In [46]:
paragraph_dataset

,index,paragraph
0,0,과거에 비해 다소 살이 빠진 듯한 방시혁 하이브 의장의 모습이 소셜미디어(sns) ...
1,0,방 의장은 전날 자신의 인스타그램 계정에 진과 함께 찍은 사진을 공개하며 “성공적인...
2,0,"이날 개최한 팬 이벤트는 진의 전역 후 첫 행사이자, bts의 데뷔 11주년 행사였..."
3,0,방시혁 하이브 의장이 지난달 28일 오후 무함마드 빈 자예드 알 나흐얀 아랍에미리트...
4,0,줄곧 침묵을 유지하던 방 의장은 지난달 17일 법원에 제출한 탄원서를 통해 “한 사...
...,...,...
33648,6435,이 대표는 경남 창원 민주당 경남도당에서 열린 현장 선거대책위원회에서 같은 당 김경...
33649,6435,이 대표는 전날 자신이 내놓은 ‘국민 1인당 25만원씩(총 13조원 추산) 민생회복...
33650,6435,"국민의힘은 세 자녀 등록금 면제 대상은 34만명이고, 들어갈 예산은 1조4500억원..."
33651,6435,"세 자녀 가구에 지원되는 전기요금, 도시가스, 지역난방비 감면을 두 자녀 가구로 확..."


In [47]:
paragraph_embedding_dataset=paragraph_embedding_dataset[paragraph_dataset['index'].isin(similar_index_list)] 
paragraph_dataset=paragraph_dataset[paragraph_dataset['index'].isin(similar_index_list)]

In [48]:
paragraph_dataset

,index,paragraph
2146,434,하이브와 경영권 분쟁 중인 민희진 어도어 대표가 자신을 응원해주는 뉴진스 팬덤 ‘버...
2147,434,아울러 “이 감사함을 다 어떻게 표현해야 할지 모르겠다. 그분들 덕분에 이상한 선택...
2148,434,기자회견 말미에는 “변호사 수임료를 내서 지금은 현금이 없는데 100억원 이상은 사...
2149,434,하이브는 가처분 인용 후 밝힌 입장대로 어도어 임시 주주총회에서 민 대표 해임에 대...
2150,434,민 대표 측 인사인 기존 어도어 사내이사 신모 부대표와 김모 이사는 해임했다. 이로...
...,...,...
33208,6376,이 대표는 윤 대통령과 전화통화를 나눈 지난 19일 자신의 유튜브로 중계한 ‘당원과...
33209,6376,최상목 경제부총리 겸 기획재정부 장관도 18일(현지시간) 제2차 주요 20개국(g2...
33210,6376,채상병 특검법의 경우 총선 직후 국민의힘에서도 “찬성표를 던질 계획(안철수 의원)”...
33211,6376,"대통령실의 의제를 놓고 여권에선 국무총리 인준과 의정 갈등 해법 논의, 중대재해처벌..."


In [49]:
paragraph_embedding_dataset

array([[-0.2956844 ,  0.24776833, -0.22421744, ..., -0.2747291 ,
        -0.3685873 , -0.1613878 ],
       [-0.17203742, -0.08865188, -0.03015941, ..., -0.04877322,
        -0.40851593, -0.14290863],
       [-0.32551047,  0.00878766,  0.14862892, ..., -0.01757442,
        -0.3642966 , -0.0799266 ],
       ...,
       [-0.25395983, -0.15503576, -0.09961025, ..., -0.25965226,
        -0.40697244,  0.02721978],
       [-0.08125061, -0.06931539, -0.24182789, ...,  0.118022  ,
        -0.39579332, -0.11885818],
       [-0.22527277, -0.1938359 ,  0.03289722, ..., -0.37068295,
        -0.2277479 , -0.04157471]], dtype=float32)

In [50]:
target_embeddings = model.encode(target_paragraph_data['paragraph'].tolist())  # 현재 읽고 있는 기사 단락 임베딩

In [51]:
target_embeddings.shape

(6, 128)

#### 현재 읽고 있는 기사 데이터와 유사한 기사 데이터를 합쳐서 훈련 데이터로 들어감

In [52]:
train_paragraph_embeddings=np.vstack((target_embeddings, paragraph_embedding_dataset))
train_paragraph_data = pd.concat([target_paragraph_data, paragraph_dataset], axis=0)

#### BERTopic을 이용한 클러스터링

In [66]:
model = BERTopic(embedding_model='bongsoo/kpf-sbert-128d-v1', min_topic_size=5)

In [67]:
topics, probs = model.fit_transform(documents=train_paragraph_data['paragraph'], embeddings=train_paragraph_embeddings)  # 클러스터링 만들기
train_paragraph_data['topic'] = topics

In [68]:
# 현재 읽고 있는 기사의 토픽 모델링
target_paragraph_data = pd.merge(target_paragraph_data, train_paragraph_data[['paragraph', 'topic']], on='paragraph', how='inner')

In [69]:
# 토픽이 -1, 0은 제외
target_paragraph_data = target_paragraph_data[target_paragraph_data['topic'] > 0]

KeyError: 'topic'

#### 결과 출력을 위한 과정

In [1]:
if len(target_paragraph_data)==0: # 만약 토픽이 없다면
    print('No Topic')
    various_news_index=similar_index_list # 유사한 기사 3개 출력

else:
    paragraph_dataset=pd.merge(paragraph_dataset, train_paragraph_data[['paragraph','topic']], on='paragraph', how='inner')
    paragraph_dataset=paragraph_dataset[paragraph_dataset['topic']>0]

    topic_embeddings=model.topic_embeddings_
    topic_embeddings=topic_embeddings[1:]

    target_topic = target_paragraph_data['topic'].value_counts().idxmax()
    target_topic_embedding = topic_embeddings[target_topic]

    num_topics = len(model.get_topic_freq()) - 1

    # faiss를 이용해서 토픽 간 코사인 유사도 계산
    index = faiss.IndexFlatIP(128)
    faiss.normalize_L2(topic_embeddings)
    index.add(topic_embeddings)
    distances, indices = index.search(np.expand_dims(target_topic_embedding, axis=0), num_topics)

    # 가장 유사도가 낮은 토픽 순으로 단락 정렬
    indices = indices[0][::-1]
    indices = np.delete(indices, np.where(indices == 0)[0][0])
    paragraph_dataset['topic'] = pd.Categorical(paragraph_dataset['topic'], categories=indices, ordered=True)
    paragraph_dataset = paragraph_dataset.sort_values('topic')



    # 토픽이 3개 이상이면
    if num_topics - 2 > 3:
        index_counts = paragraph_dataset.groupby(
            'topic')['index'].value_counts().rename('count').reset_index()
        most_common_index_per_topic = index_counts.loc[index_counts.groupby('topic')[
            'count'].idxmax()]
        most_common_index_per_topic=most_common_index_per_topic.drop_duplicates(subset='index') # 중복 제거

        various_news_index=most_common_index_per_topic['index'].tolist()

    else: # 토픽이 3개 이하이면 나온 것 모두 반환
        paragraph_dataset=paragraph_dataset.drop_duplicates(subset='index') #중복 제거
        various_news_index=paragraph_dataset['index'].tolist()

NameError: name 'target_paragraph_data' is not defined

#### 결과 확인

In [ ]:
various_news_index 

In [ ]:
if target_link in news_dataset['link']:
    same_news_index=news_dataset[news_dataset['link']==target_link].index
    various_news_index.remove(same_news_index)

In [ ]:
various_news=news_dataset.loc[various_news_index][:3]

In [ ]:
various_news